In [27]:
import json
import pandas as pd
import numpy as np
import os
import glob
import codecs
import datetime
today_foloder="20180405" # str(datetime.datetime.now().year)+str(datetime.datetime.now().month).zfill(2)+str(datetime.datetime.now().day).zfill(2)
date1="2018-03-31" # Week ending Saturday
# date2="2018-02-17" # Week ending Saturday, Not available for 5 days prediction
# https://openweathermap.org/weather-conditions reference of weather group

json_list_date=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/*.json")

In [28]:
json_date1_Friday=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+
                                 str(datetime.datetime.strptime(date1,"%Y-%m-%d").date()+datetime.timedelta(days=-1))
                                 +".json",'r'))
json_date1_Saturday=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+
                                 str(datetime.datetime.strptime(date1,"%Y-%m-%d").date())
                                 +".json",'r'))
json_date1_Sunday=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+
                                 str(datetime.datetime.strptime(date1,"%Y-%m-%d").date()+datetime.timedelta(days=-6))
                                 +".json",'r'))
json_date1_Pred_5_days_ago=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/forecasgt/forcast_api_response_"+today_foloder+"/"+
                                 str(datetime.datetime.strptime(date1,"%Y-%m-%d").date()+datetime.timedelta(days=-5))
                                 +".json",'r'))


In [29]:
sales=pd.read_excel("/home/jian/BiglotsCode/outputs/Output_"+date1+"/wide_sales_date"+date1+".xlsx",sheetname="sales")

In [30]:
date1_ly=str(datetime.datetime.strptime(date1,"%Y-%m-%d").date()+datetime.timedelta(days=-364))
sales=sales[["location_id",date1,date1_ly]]

In [31]:
store_dma=pd.read_excel("/home/jian/Projects/Big_Lots/Other Input/all_store_DMA.xlsx",dtype=str)
store_dma=store_dma[store_dma['zip_cd']!='nan']
store_dma=store_dma[['location_id','zip_cd']]
store_dma=store_dma.drop_duplicates().reset_index()
del store_dma['index']
store_dma['location_id']=store_dma['location_id'].astype(int)
sales=pd.merge(sales,store_dma,on="location_id",how="left")

# Weather Group Level

In [32]:
response=json_date1_Pred_5_days_ago
all_weather_groups=[]
all_weather_desc=[]
all_weather_id=[]
for zip_cd in list(response.keys()):
    weather_group = []
    weather_desc = []
    weather_id = []
    for i in range(len(response[zip_cd]['list'][39]['weather'])):
        weather_group=list(set(weather_group+[response[zip_cd]['list'][39]['weather'][i]['main']]))
        all_weather_groups=list(set(all_weather_groups+weather_group))
        
        weather_desc=list(set(weather_desc+[response[zip_cd]['list'][39]['weather'][i]['description']]))
        all_weather_desc=list(set(all_weather_desc+weather_desc))

        weather_id=list(set(weather_id+[response[zip_cd]['list'][39]['weather'][i]['id']]))
        all_weather_id=list(set(all_weather_id+weather_id))
sales_included=sales[(sales[date1]!=0) & (sales[date1_ly]!=0)]
sales_excluded=sales[(sales[date1]==0) | (sales[date1_ly]==0)]
sales_included.reset_index(inplace=True)
del sales_included['index']
sales_excluded.reset_index(inplace=True)

In [33]:
sales_included['YoY_Pctg']=(sales_included[date1]-sales_included[date1_ly])/sales_included[date1_ly]
sales_included['YoY_Dollar']=sales_included[date1]-sales_included[date1_ly]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
df=pd.DataFrame()
df_date=pd.DataFrame()

weather_list=[]
weather_code_list=[]
weather_desc_list=[]

df_weather_agg=pd.DataFrame()
for zip_cd in list(response.keys()):
    temp_max=response[zip_cd]['list'][39]['main']['temp_max'] * 9/5 - 459.67    
    humidity=response[zip_cd]['list'][39]['main']['humidity']/100    
    pressure=response[zip_cd]['list'][39]['main']['pressure']
    wind_speed=response[zip_cd]['list'][39]['wind']['speed']
    if 'deg' in response[zip_cd]['list'][39]['wind'].keys():
        wind_deg=response[zip_cd]['list'][39]['wind']['deg']
    else:
        wind_deg=np.nan
        
    df_weather_binary=pd.DataFrame(columns=all_weather_groups,index=[0])
    df_weather_binary=df_weather_binary.fillna(0)
    for k in range(len(response[zip_cd]['list'][39]['weather'])):
        for weather_group in all_weather_groups:
            if response[zip_cd]['list'][39]['weather'][k]['main']==weather_group:
                df_weather_binary[weather_group]=1
    time=str(datetime.datetime.fromtimestamp(response[zip_cd]['list'][39]['dt']).time())
    date=str(datetime.datetime.fromtimestamp(response[zip_cd]['list'][39]['dt']).date())
    weather = []
    weather_types=len(response[zip_cd]['list'][39]['weather'])
    
    for j in range(weather_types):
        weather = weather+[str(response[zip_cd]['list'][39]['weather'][j]['main'])]
    
    country=response[zip_cd]['city']['country']
    name=response[zip_cd]['city']['name']
    
    if 'clouds' in response[zip_cd]['list'][39].keys():
        clouds=response[zip_cd]['list'][39]['clouds']['all']/100
    else: 
        clouds= float('NaN')
            
    df_weather=pd.DataFrame([{'date':date,'time':time,'zip_cd':zip_cd,'city_name_weather':name,
                             'weather':weather,'weather_types':weather_types,'Cloudiness':clouds,'humidity':humidity,'pressure':pressure,
                              'temp_max':temp_max,'wind_speed':wind_speed}])
    df_weather=df_weather.join(df_weather_binary)
    df_weather_agg=df_weather_agg.append(df_weather,ignore_index=False)
    
sales_included_group=pd.merge(sales_included,df_weather_agg,on="zip_cd",how="left")
# sales_included_group.to_csv("/home/jian/Projects/Big_Lots/Weather/output_for_sales_model/Group_"+date1+"_5_Days_Predicted.csv",index=False)

In [35]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Weather/output_for_sales_model/Excel_Group_"+date1+"_5_Days_Predicted.xlsx",engine="xlsxwriter")
sales_included_group.to_excel(writer,"data",index=False)
for weather_group in all_weather_groups:
    df=sales_included_group[['YoY_Pctg','YoY_Dollar',weather_group]]
    df=df[~pd.isnull(df[weather_group])]
    df_1=df[df[weather_group]==1]
    mean_1_pctg=df_1['YoY_Pctg'].mean()
    mean_1_dollar=df_1['YoY_Dollar'].mean()
    
    df_0=df[df[weather_group]==0]
    mean_0_pctg=df_0['YoY_Pctg'].mean()
    mean_0_dollar=df_0['YoY_Dollar'].mean()

    df=pd.DataFrame({weather_group:[1,0],"YoY_Pctg_Mean":[mean_1_pctg,mean_0_pctg],"YoY_Dollar_Mean":[mean_1_dollar,mean_0_dollar]})
    df=df[[weather_group,"YoY_Pctg_Mean","YoY_Dollar_Mean"]]
    df.to_excel(writer,weather_group,index=False)
writer.save()

# Description Level

In [36]:
df=pd.DataFrame()
df_date=pd.DataFrame()

weather_list=[]
weather_code_list=[]
weather_desc_list=[]

df_weather_agg=pd.DataFrame()
for zip_cd in list(response.keys()):
    temp_max=response[zip_cd]['list'][39]['main']['temp_max'] * 9/5 - 459.67    
    humidity=response[zip_cd]['list'][39]['main']['humidity']/100    
    pressure=response[zip_cd]['list'][39]['main']['pressure']
    wind_speed=response[zip_cd]['list'][39]['wind']['speed']
    if 'deg' in response[zip_cd]['list'][39]['wind'].keys():
        wind_deg=response[zip_cd]['list'][39]['wind']['deg']
    else:
        wind_deg=np.nan
        
    df_weather_binary=pd.DataFrame(columns=all_weather_desc,index=[0])
    df_weather_binary=df_weather_binary.fillna(0)
    for k in range(len(response[zip_cd]['list'][39]['weather'])):
        for weather_group in all_weather_desc:
            if response[zip_cd]['list'][39]['weather'][k]['description']==weather_group:
                df_weather_binary[weather_group]=1
    time=str(datetime.datetime.fromtimestamp(response[zip_cd]['list'][39]['dt']).time())
    date=str(datetime.datetime.fromtimestamp(response[zip_cd]['list'][39]['dt']).date())
    weather = []
    weather_types=len(response[zip_cd]['list'][39]['weather'])
    
    for j in range(weather_types):
        weather = weather+[str(response[zip_cd]['list'][39]['weather'][j]['description'])]
    
    country=response[zip_cd]['city']['country']
    name=response[zip_cd]['city']['name']
    
    if 'clouds' in response[zip_cd]['list'][39].keys():
        clouds=response[zip_cd]['list'][39]['clouds']['all']/100
    else: 
        clouds= float('NaN')
            
    df_weather=pd.DataFrame([{'date':date,'time':time,'zip_cd':zip_cd,'city_name_weather':name,
                             'weather':weather,'weather_types':weather_types,'Cloudiness':clouds,'humidity':humidity,'pressure':pressure,
                              'temp_max':temp_max,'wind_speed':wind_speed}])
    df_weather=df_weather.join(df_weather_binary)
    df_weather_agg=df_weather_agg.append(df_weather,ignore_index=False)
    
sales_included_desc=pd.merge(sales_included,df_weather_agg,on="zip_cd",how="left")
# sales_included_desc.to_csv("/home/jian/Projects/Big_Lots/Weather/output_for_sales_model/Desc_"+date1+"_5Days_Predicted.csv",index=False)

In [37]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Weather/output_for_sales_model/Excel_Desc_"+date1+"_5_Days_Predicted.xlsx",engine="xlsxwriter")
sales_included_desc.to_excel(writer,"data",index=False)
for weather_desc in all_weather_desc:
    df=sales_included_desc[['YoY_Pctg','YoY_Dollar',weather_desc]]
    df=df[~pd.isnull(df[weather_desc])]
    df_1=df[df[weather_desc]==1]
    mean_1_pctg=df_1['YoY_Pctg'].mean()
    mean_1_dollar=df_1['YoY_Dollar'].mean()
    
    df_0=df[df[weather_desc]==0]
    mean_0_pctg=df_0['YoY_Pctg'].mean()
    mean_0_dollar=df_0['YoY_Dollar'].mean()

    df=pd.DataFrame({weather_desc:[1,0],"YoY_Pctg_Mean":[mean_1_pctg,mean_0_pctg],"YoY_Dollar_Mean":[mean_1_dollar,mean_0_dollar]})
    df=df[[weather_desc,"YoY_Pctg_Mean","YoY_Dollar_Mean"]]
    df.to_excel(writer,weather_desc,index=False)
writer.save()